In [1]:
from etl import open_server_users
from etl import open_server_engage
from etl import open_server_payment
from etl import open_server_battle
from etl import open_server_resource
from etl import server_open_nd
import numpy as np
from config import ETLConfig

### 1. Open server settings

1. Define the game you want to analyze
2. Define days after server open you want data from
3. Define server_ids

In [2]:
from camel_utils_x.camel_sql import SQLBase

In [3]:
game = 'aoz'

days = 10

server_ids = [123,234,5,6,7,8,9]

### 2. Define server open
Fields in returned dataframe will be used in making queries to each server

In [4]:
# unshifted version of server open
server_open_nd()

,open_time,open_date,open_10_date,open_10_date_id,open_date_id
server_id,,,,,
0,2020-08-10,2020-08-10,2020-08-20,20200820,20200810
1,2018-09-20,2018-09-20,2018-09-30,20180930,20180920
2,2018-10-25,2018-10-25,2018-11-04,20181104,20181025
3,2018-11-16,2018-11-16,2018-11-26,20181126,20181116
4,2018-11-18,2018-11-18,2018-11-28,20181128,20181118
...,...,...,...,...,...
348,2020-11-18,2020-11-18,2020-11-28,20201128,20201118
349,2020-11-20,2020-11-20,2020-11-30,20201130,20201120
350,2020-11-22,2020-11-22,2020-12-02,20201202,20201122


In [5]:
# shifted version of sever open
server_open_nd(offset = 10, days = 20)

,open_time,open_date,open_20_date,open_20_date_id,open_date_id
server_id,,,,,
0,2020-08-20,2020-08-20,2020-09-09,20200909,20200820
1,2018-09-30,2018-09-30,2018-10-20,20181020,20180930
2,2018-11-04,2018-11-04,2018-11-24,20181124,20181104
3,2018-11-26,2018-11-26,2018-12-16,20181216,20181126
4,2018-11-28,2018-11-28,2018-12-18,20181218,20181128
...,...,...,...,...,...
348,2020-11-28,2020-11-28,2020-12-18,20201218,20201128
349,2020-11-30,2020-11-30,2020-12-20,20201220,20201130
350,2020-12-02,2020-12-02,2020-12-22,20201222,20201202


### Making queries

*On User*

In [6]:
user = open_server_users(server_ids = [234],days = 5, offset = 3, fields = ['id','create_time','language'])

UserType: 234: 100%|████████████████████████████████████████| 1/1 [00:06<00:00,  6.44s/it]


**From this line, we see that server 234 is opened on 2020-5-3**

In [7]:
server_open_nd(game).loc[234]

open_time          2020-05-03 00:00:00
open_date                   2020-05-03
open_10_date                2020-05-13
open_10_date_id               20200513
open_date_id                  20200503
Name: 234, dtype: object

**Since we shifted 3 days, the start date wound be 2020-5-6**         
**Setting days = 5, enables stopping at 2020-5-11 (excluded)**

In [8]:
server_open_nd(game, offset = 3, days = 5).loc[234]

open_time         2020-05-06 00:00:00
open_date                  2020-05-06
open_5_date                2020-05-11
open_5_date_id               20200511
open_date_id                 20200506
Name: 234, dtype: object

**Check dates in dataframe**

In [23]:
sorted(user.date.dt.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*On engage / with same days and offset configurations*

In [10]:
engage = open_server_engage(server_ids = [234], days = 5, offset = 3)

Engagement 234: 100%|███████████████████████████████████████| 1/1 [00:01<00:00,  1.93s/it]


In [11]:
sorted(engage.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

**If a server open dataframe is  passed into query functions, no additional open server queries execution inside queryies**

In [18]:
server_open = server_open_nd(game = game, offset = 3, days = 5)

user = open_server_users(server_ids = [234], fields = ['id','create_time','language'], 
                         server_open = server_open, days = 5,offset = 3)

engage = open_server_engage(server_ids = [234], server_open = server_open, days = 5, offset = 3)

payment = open_server_payment(server_ids = [234], server_open = server_open, days = 5, offset = 3)

battle = open_server_battle(server_ids = [234], server_open = server_open, days = 5, offset = 3)

resource = open_server_resource(server_ids = [234], server_open = server_open, days = 5, offset = 3)

Resource 234: 100%|█████████████████████████████████████████| 1/1 [00:02<00:00,  2.19s/it]


*Check user table*

In [24]:
sorted(user.date.dt.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check engage table*

In [14]:
sorted(engage.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check payment table*

In [15]:
payment.date.dt.strftime('%Y-%m-%d').unique()

array(['2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09',
       '2020-05-10'], dtype=object)

*Check battle table*

In [16]:
sorted(battle.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

*Check resource table*

In [21]:
sorted(resource.date.unique())

[datetime.date(2020, 5, 6),
 datetime.date(2020, 5, 7),
 datetime.date(2020, 5, 8),
 datetime.date(2020, 5, 9),
 datetime.date(2020, 5, 10)]

------------
------------